In [1]:

exec(open("./funcs/tool_funcs.py").read())




In [2]:
def cal_need_update_list(remove_node, X_APSP_matrix):
    N = len(X_APSP_matrix)
    removal = remove_node
    remaining_list = [i for i in np.arange(N) if i != removal]
    need_update_list = [[] for i in range(N)]
    how_many_counter = np.zeros(N)
    for i in remaining_list:
        for j in remaining_list:
            if i != j:
                if X_APSP_matrix[i,j] >= X_APSP_matrix[i,removal] + X_APSP_matrix[removal,j]:
                    need_update_list[i].append(j)
                    how_many_counter[i] += 1
    
    how_many = np.sum(how_many_counter > 0)
    
    return need_update_list, how_many

def generate_new_distance_matrix(X_distance_matrix, remove_node):
    X_distance_matrix_new = X_distance_matrix.copy()
 

    N = len(X_distance_matrix)
    for i in range(N):
        for j in range(N):
            if i == remove_node or j == remove_node:
                X_distance_matrix_new[i,j] = np.inf

    X_distance_matrix_new[remove_node,remove_node] = 0  
    
    return X_distance_matrix_new

In [3]:
def dijkstra_one_to_all(distance_matrix, src, need_up_i, remove_node):
    
    N = len(distance_matrix)
    dist = [np.inf] * N
    dist[src] = 0
    visited = [False] * N

    # Priority queue: (distance, node)
    pq = [(0, src)]
    
    kk = len(need_up_i)
    counter = 0
    visited[remove_node] = True

    while pq:
        current_dist, u = heapq.heappop(pq)

        if visited[u]:
            continue
        visited[u] = True
        
        if u in need_up_i:
            counter += 1
            if counter == kk:
                # import pdb;pdb.set_trace()
                return np.array(dist)
 

        # Update distances for neighbors
        for v in range(N):
            if not visited[v] and distance_matrix[u][v] > 0:
                new_dist = current_dist + distance_matrix[u][v]
                if new_dist < dist[v]:
                    dist[v] = new_dist
                    heapq.heappush(pq, (new_dist, v))

    return np.array(dist)

In [4]:

def cal_new_APSP_matrix_warm(X_distance_matrix, X_APSP_matrix, remove_node):
    N = len(X_distance_matrix)
    X_APSP_matrix_new = X_APSP_matrix.copy()
    remaining_list = [i for i in np.arange(N) if i != remove_node]

    need_update_list, how_many = cal_need_update_list(remove_node, X_APSP_matrix)
    
    Cost = np.round(how_many/(N - 1), 3)
    
    print(f"The cost for re-caculating the APSP matrix is: {Cost}.")

    X_distance_matrix_new = generate_new_distance_matrix(X_distance_matrix, remove_node)
 
    for i in remaining_list:
        if len(need_update_list[i]) > 0:
            temp = dijkstra_one_to_all(X_distance_matrix_new, i, need_update_list[i], remove_node)

            for j in need_update_list[i]:
                X_APSP_matrix_new[i,j] = temp[j]
 
    for i in range(N):
        for j in range(N):
            if i == remove_node or j == remove_node:
                X_APSP_matrix_new[i,j] = np.inf

    X_APSP_matrix_new[remove_node,remove_node] = 0
    
    return X_APSP_matrix_new 


In [5]:
def shortest_path_n_to_r_new(distance_matrix, shortest_path_matrix, remaining_list, r, remove_node):
  
    max_jump_list = []
    for t in remaining_list:
        m_jump =  distance_matrix[remove_node,t] + shortest_path_matrix[t,r]
        max_jump_list.append(m_jump)
    return np.min(max_jump_list)

def shortest_path_r_to_n_new(distance_matrix, shortest_path_matrix, remaining_list, r, remove_node):
    max_jump_list = []
    for t in remaining_list:
        m_jump =  shortest_path_matrix[r,t] + distance_matrix[t, remove_node] 
        max_jump_list.append(m_jump)
    return np.min(max_jump_list)

     
def cal_n_shortest_path_new(distance_matrix, shortest_path_matrix, remove_node, remaining_list):
    for r in remaining_list:
        shortest_path_matrix[remove_node, r] = shortest_path_n_to_r_new(distance_matrix, shortest_path_matrix, remaining_list, r, remove_node)
        shortest_path_matrix[r, remove_node] = shortest_path_r_to_n_new(distance_matrix, shortest_path_matrix, remaining_list, r, remove_node)
        
    for i in remaining_list:        
        for j in remaining_list:
            if i < j:
                shortest_path_matrix[i,j] =  update_shortest_path_ij_new(distance_matrix, shortest_path_matrix, remove_node, i, j)
                shortest_path_matrix[j,i] =  update_shortest_path_ij_new(distance_matrix, shortest_path_matrix, remove_node, j, i)
                
def update_shortest_path_ij_new(distance_matrix, shortest_path_matrix, remove_node, i,j):
    m1 = shortest_path_matrix[i,j]
    m2 =  shortest_path_matrix[i,remove_node] + shortest_path_matrix[remove_node,j] 
    return np.min((m1,m2))



In [6]:
def cal_which_to_remove(X_APSP_matrix, edge_nodes):
   
    _, how_many0 = cal_need_update_list(edge_nodes[0], X_APSP_matrix)
    _, how_many1 = cal_need_update_list(edge_nodes[1], X_APSP_matrix)
    if how_many0 < how_many1:
        remove_node = edge_nodes[0]
    else:
        remove_node = edge_nodes[1]
    # print(how_many0, how_many1, edge_nodes, remove_node)
    return remove_node
    
    
    

In [7]:

def cal_APSP_matrix_after_graph_update(X_distance_matrix, X_APSP_matrix, edge_nodes, X_distance_matrix_updated):
    
    N = len(X_distance_matrix)
    
    remove_node = cal_which_to_remove(X_APSP_matrix, edge_nodes)

    X_APSP_matrix_updated = cal_new_APSP_matrix_warm(X_distance_matrix, X_APSP_matrix, remove_node)
 
    remaining_list = [i for i in np.arange(N) if i != remove_node]
    
    cal_n_shortest_path_new(X_distance_matrix_updated, X_APSP_matrix_updated, remove_node, remaining_list)
    
    return X_APSP_matrix_updated

 
    

In [8]:
# N = 100

# X_distance_matrix = create_distance_matrix(N)

In [9]:
X_distance_matrix = np.loadtxt("./data/X_100_distance_matrix.csv", delimiter=",")
 

In [10]:
edge_nodes = [3, 9]

edge_weight = 50    

X_distance_matrix_new = X_distance_matrix.copy()
X_distance_matrix_new[edge_nodes[0],edge_nodes[1]] = edge_weight 

X_APSP_matrix = floyd_warshall(X_distance_matrix)

In [11]:
 

start = time.time()
X_APSP_matrix_new_warm = cal_APSP_matrix_after_graph_update(X_distance_matrix, X_APSP_matrix, edge_nodes, X_distance_matrix_new)
end = time.time()


time_used1_warm = end - start
time_used1_warm = np.round(time_used1_warm, 3)

The cost for re-caculating the APSP matrix is: 0.434.


In [12]:

print(f"Time used: {time_used1_warm}s" )

Time used: 0.224s


In [13]:
 
start = time.time()
X_APSP_matrix_new_floyd_warshall = floyd_warshall(X_distance_matrix_new)
end = time.time()


time_used2_floyd = end - start
time_used2_floyd = np.round(time_used2_floyd, 3)

In [14]:
print(f"Time used: {time_used2_floyd}s" )

Time used: 1.015s


In [15]:
print(np.allclose(X_APSP_matrix_new_warm, X_APSP_matrix_new_floyd_warshall))

True


In [16]:
print(f"Ratio: {np.round(time_used1_warm/time_used2_floyd, 2)}")


Ratio: 0.22


In [17]:
X_APSP_matrix_new_warm[1]


array([31.,  0., 63., 23., 56., 57., 59., 41., 46., 56., 33., 26., 25.,
       20., 27., 23., 49., 48., 33., 42., 35., 47., 48., 41., 36., 37.,
       30., 48., 38., 28., 55., 22., 60., 24., 28., 55., 60., 24., 43.,
       85., 25., 47., 16., 65., 48., 50., 10., 82., 34., 44., 59., 15.,
        8., 46., 54., 39., 40., 45., 49., 32., 19., 48., 68., 27., 85.,
       54., 76., 48., 53., 17., 15., 32., 30., 34., 52., 31., 17., 55.,
       58., 70., 42., 64., 53., 40., 52., 20., 50., 63., 42., 40., 48.,
       24., 65., 53., 39., 31., 29., 32., 26., 17.])